# 동적 계획법

동적 계획법은 복잡한 문제를 재귀를 통해 간단한 하위 문제로 분류하여 단순화하여 해결하는 방법이다. 문제가 최적 부분 구조와 중복되는 부분 문제를 가지고 있다면, 동적 계획법으로 해결할 수 있다. 

최적 부분 구조는 답을 구하기 위해서 했던 계산을 반복해야 하는 문제의 구조를 말한다. 동적 계획법을 사용하려면 먼저 최적 부분 구조가 있는지 확인해야 한다. 동적 계획법은 부분 문제를 풀고 결과를 저장한 후, 다음 부분 문제(중복되는 부분 문제)를 푸는 과정에서 저당된 결과를 사용한다 

## 11.1 메모이제이션
메모이제이션은 프로그램이 동일한 계산을 반복할 때, 이전에 계산한 값을 메모리에 저장하여 동일한 계산의 반복 수행을 제거하여 프로그램의 실행 속도를 빠르게 하는 기법읻가. 

### 11.1.1 피보나치 수열

In [7]:
# 시간 측정 함수
from functools import wraps
import time

def benchmark(method):
    @wraps(method)
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        print('{0}:{1:0.2f} ms'.format(method.__name__,((te-ts)*1000)))
        return result
    return timed

In [8]:
from functools import wraps

def memo(func):
    cache = {}
    
    @wraps(func)
    def wrap(*args):
        if args not in cache:
            cache[args] = func(*args)
        return cache[args]
    return wrap

def fib(n):
    if n < 2:
        return 1
    else:
        return fib(n-1) + fib(n-2)

@memo
def fib2(n):
    if n < 2:
        return 1
    else:
        return fib2(n-1) + fib2(n-2)

def fib3(m,n):
    if m[n] == 0:
        m[n] = fib3(m,n-1) + fib3(m,n-2)
    return m[n]

@benchmark
def test_fib(n):
    print(fib(n))

@benchmark
def test_fib2(n):
    print(fib2(n))

@benchmark
def test_fib3(n):
    m = [0] * (n+1)
    m[0], m[1] = 1,1
    print(fib3(m,n))

if __name__ == '__main__':
    n = 35
    test_fib(n)
    test_fib2(n)
    test_fib3(n)
    
    


14930352
test_fib:3208.47 ms
14930352
test_fib2:0.09 ms
14930352
test_fib3:0.03 ms


## 11.2 연습문제
### 11.2.1 최장 증가 부분멸
증가하는 순서대로(오름차순) 숫자를 고른 부분열의 길이가 최대가 되게 하면 된다. 

In [10]:
from bisect import bisect
from itertools import combinations
from functools import wraps

# 가장 단순한 방식
def naive_longest_inc_subseq(seq):
    for length in range(len(seq), 0, -1):
        for sub in combinations(seq, length):
            if list(sub) == sorted(sub):
                return len(sub)

# 동적 계획법
def dp_longest_inc_subseq(seq):
    L = [1] * len(seq)
    res = []
    for cur, val in enumerate(seq):
        for pre in range(cur):
            if seq[pre] <= val:
                L[cur] = max(L[cur], 1+ L[pre])
    return max(L)

def memo(func):
    cache = {}
    
    @wraps(func)
    def wrap(*args):
        if args not in cache:
            cache[args] = func(*args)
        return cache[args]
    return wrap

# 메모이제이션
def memoized_longest_inc_subseq(seq):
    @memo
    def L(cur):
        res = 1
        for pre in range(cur):
            if seq[pre] <= seq[cur]:
                res = max(res,1+L(pre))
        return res
    return max(L(i) for i in range(len(seq)))

# 이진 검색
def longest_inc_bisec(seq):
    end = []
    for val in seq:
        idx = bisect(end, val)
        if idx == len(end):
            end.append(val)
        else:
            end[idx] = val
    return len(end)

@benchmark
def test_naive_longest_inc_subseq():
    print(naive_longest_inc_subseq(s1))

@benchmark
def test_dp_longest_inc_subseq():
    print(dp_longest_inc_subseq(s1))

@benchmark
def test_memoized_longest_inc_subseq():
    print(memoized_longest_inc_subseq(s1))

@benchmark
def test_longest_inc_bisec():
    print(longest_inc_bisec(s1))

if __name__ == '__main__':
    s1 = [94,8,78,22,38,79,93,8,84,39]
    print(s1)
    test_naive_longest_inc_subseq()
    test_dp_longest_inc_subseq()
    test_memoized_longest_inc_subseq()
    test_longest_inc_bisec()
    

[94, 8, 78, 22, 38, 79, 93, 8, 84, 39]
5
test_naive_longest_inc_subseq:0.27 ms
5
test_dp_longest_inc_subseq:0.03 ms
5
test_memoized_longest_inc_subseq:0.05 ms
5
test_longest_inc_bisec:0.03 ms
